# Import and process 《全唐诗》in preparation for training 

In [1]:
import nltk
import sys
import os
import re
import numpy             as np
import matplotlib.pyplot as plt
import cPickle           as pickle

#### Look at the content in txt file

In [2]:
filename = '全唐诗.txt'

In [4]:
f = open(filename, 'rb')
for i in range(0, 1000):
    line = unicode(f.readline(), "cp936")
    print len(line)
    print line
f.close()

8
全唐诗 卷一

2


18
　　卷1_1【帝京篇十首】李世民

2


26
秦川雄帝宅，函谷壮皇居。绮殿千寻起，离宫百雉馀。

26
连薨遥接汉，飞观迥凌虚。云日隐层阙，风烟出绮疏。

2


26
岩廊罢机务，崇文聊驻辇。玉匣启龙图，金绳披凤篆。

26
韦编断仍续，缥帙舒还卷。对此乃淹留，欹案观坟典。

2


26
移步出词林，停舆欣武宴。雕弓写明月，骏马疑流电。

26
惊雁落虚弦，啼猿悲急箭。阅赏诚多美，于兹乃忘倦。

2


26
鸣笳临乐馆，眺听欢芳节。急管韵朱弦，清歌凝白雪。

26
彩凤肃来仪，玄鹤纷成列。去兹郑卫声，雅音方可悦。

2


26
芳辰追逸趣，禁苑信多奇。桥形通汉上，峰势接云危。

26
烟霞交隐映，花鸟自参差。何如肆辙迹，万里赏瑶池。

2


26
飞盖去芳园，兰桡游翠渚。萍间日彩乱，荷处香风举。

26
桂楫满中川，弦歌振长屿。岂必汾河曲，方为欢宴所。

2


26
落日双阙昏，回舆九重暮。长烟散初碧，皎月澄轻素。

26
搴幌玩琴书，开轩引云雾。斜汉耿层阁，清风摇玉树。

2


26
欢乐难再逢，芳辰良可惜。玉酒泛云罍，兰殽陈绮席。

26
千钟合尧禹，百兽谐金石。得志重寸阴，忘怀轻尺璧。

2


26
建章欢赏夕，二八尽妖妍。罗绮昭阳殿，芬芳玳瑁筵。

26
佩移星正动，扇掩月初圆。无劳上悬圃，即此对神仙。

2


26
以兹游观极，悠然独长想。披卷览前踪，抚躬寻既往。

26
望古茅茨约，瞻今兰殿广。人道恶高危，虚心戒盈荡。

26
奉天竭诚敬，临民思惠养。纳善察忠谏，明科慎刑赏。

26
六五诚难继，四三非易仰。广待淳化敷，方嗣云亭响。

2


17
卷1_2【饮马长城窟行】李世民

2


26
塞外悲风切，交河冰已结。瀚海百重波，阴山千里雪。

26
迥戍危烽火，层峦引高节。悠悠卷旆旌，饮马出长城。

26
寒沙连骑迹，朔吹断边声。胡尘清玉塞，羌笛韵金钲。

26
绝漠干戈戢，车徒振原隰。都尉反龙堆，将军旋马邑。

26
扬麾氛雾静，纪石功名立。荒裔一戎衣，灵台凯歌入。

2


16
卷1_3【执契静三边】李世民

2


26
执契静三边，持衡临万姓。玉彩辉关烛，金华流日镜。

26
无为宇宙清，有美璇玑正。皎佩星连景，飘衣云结庆。

26
戢武耀七德，升文辉九功。烟波澄旧碧，尘火息前红。



## Construct training data

In [5]:
# Test some special characters
special = [u'\u3010', u'\u3011'] ##'【' & '】'
f = open(filename, 'rb')
for i in range(0, 3):
    line = unicode(f.readline(), "cp936")
f.close()
print line
print line[12]
print re.search(u'\u3010', line).group(0)

　　卷1_1【帝京篇十首】李世民

】
【


In [6]:
def add_lines(old_line, line, poems):
    #print line
    if len(line) < 12 or re.search(u'\u3010', line) is not None:
        '''If this line is blank or title'''
        old_line = ''
    elif old_line == '':
        '''If start a new line'''
        poems.append(line[0:-2])
        old_line = line
    else:
        '''If continuing to last poem'''
        poems[-1] = poems[-1]+line[0:-2]
        old_line = line
    return poems, old_line

In [7]:
print "Reading txt file..."
poems = []
old_line = ''
with open(filename, 'rb') as f:
    while True:
        try:
            line = unicode(f.readline(), "cp936")
        except:
            continue
            
        if not line:
            break
            
        poems, old_line = add_lines(old_line, line, poems)

Reading txt file...


In [8]:
len(poems)

48975

In [9]:
# Print some examples
for i in range(0, 10):
    print poems[i]

秦川雄帝宅，函谷壮皇居。绮殿千寻起，离宫百雉馀。连薨遥接汉，飞观迥凌虚。云日隐层阙，风烟出绮疏。
岩廊罢机务，崇文聊驻辇。玉匣启龙图，金绳披凤篆。韦编断仍续，缥帙舒还卷。对此乃淹留，欹案观坟典。
移步出词林，停舆欣武宴。雕弓写明月，骏马疑流电。惊雁落虚弦，啼猿悲急箭。阅赏诚多美，于兹乃忘倦。
鸣笳临乐馆，眺听欢芳节。急管韵朱弦，清歌凝白雪。彩凤肃来仪，玄鹤纷成列。去兹郑卫声，雅音方可悦。
芳辰追逸趣，禁苑信多奇。桥形通汉上，峰势接云危。烟霞交隐映，花鸟自参差。何如肆辙迹，万里赏瑶池。
飞盖去芳园，兰桡游翠渚。萍间日彩乱，荷处香风举。桂楫满中川，弦歌振长屿。岂必汾河曲，方为欢宴所。
落日双阙昏，回舆九重暮。长烟散初碧，皎月澄轻素。搴幌玩琴书，开轩引云雾。斜汉耿层阁，清风摇玉树。
欢乐难再逢，芳辰良可惜。玉酒泛云罍，兰殽陈绮席。千钟合尧禹，百兽谐金石。得志重寸阴，忘怀轻尺璧。
建章欢赏夕，二八尽妖妍。罗绮昭阳殿，芬芳玳瑁筵。佩移星正动，扇掩月初圆。无劳上悬圃，即此对神仙。
以兹游观极，悠然独长想。披卷览前踪，抚躬寻既往。望古茅茨约，瞻今兰殿广。人道恶高危，虚心戒盈荡。奉天竭诚敬，临民思惠养。纳善察忠谏，明科慎刑赏。六五诚难继，四三非易仰。广待淳化敷，方嗣云亭响。


In [10]:
poems[0]

u'\u79e6\u5ddd\u96c4\u5e1d\u5b85\uff0c\u51fd\u8c37\u58ee\u7687\u5c45\u3002\u7eee\u6bbf\u5343\u5bfb\u8d77\uff0c\u79bb\u5bab\u767e\u96c9\u9980\u3002\u8fde\u85a8\u9065\u63a5\u6c49\uff0c\u98de\u89c2\u8fe5\u51cc\u865a\u3002\u4e91\u65e5\u9690\u5c42\u9619\uff0c\u98ce\u70df\u51fa\u7eee\u758f\u3002'

In [11]:
print len(poems[0])
poems[0][47]

48


u'\u3002'

In [13]:
# create tokenized poems
def tokenize(poem):
    # poem_tokenize = [u'poem_start']
    poem_tokenize = []
    poem_tokenize.extend([poem[i] for i in range(0, len(poem))])
    # poem_tokenize.append(u'poem_end')
    return poem_tokenize
poems_tokenize = [tokenize(poems[i]) for i in range(0, len(poems))]
print poems_tokenize[0]

[u'\u79e6', u'\u5ddd', u'\u96c4', u'\u5e1d', u'\u5b85', u'\uff0c', u'\u51fd', u'\u8c37', u'\u58ee', u'\u7687', u'\u5c45', u'\u3002', u'\u7eee', u'\u6bbf', u'\u5343', u'\u5bfb', u'\u8d77', u'\uff0c', u'\u79bb', u'\u5bab', u'\u767e', u'\u96c9', u'\u9980', u'\u3002', u'\u8fde', u'\u85a8', u'\u9065', u'\u63a5', u'\u6c49', u'\uff0c', u'\u98de', u'\u89c2', u'\u8fe5', u'\u51cc', u'\u865a', u'\u3002', u'\u4e91', u'\u65e5', u'\u9690', u'\u5c42', u'\u9619', u'\uff0c', u'\u98ce', u'\u70df', u'\u51fa', u'\u7eee', u'\u758f', u'\u3002']


In [14]:
# Check unique characters
len(np.unique(poems_tokenize))

48611

In [15]:
# Check number of characters in total
sum([len(poems_tokenize[i]) for i in range(0, len(poems_tokenize))])

3008122

In [16]:
# Plot words frequency
tokens = [item for sublist in poems_tokenize for item in sublist]
#lambda poems_tokenize: [item for sublist in poems_tokenize for item in sublist]
unique_token = np.unique(tokens)
n_unique_token = len(unique_token)
tokens_count = []
for i in range(0, n_unique_token):
    tokens_count.append(tokens.count(unique_token[i]))
#tokens.count(unique_token[0])

In [17]:
#plt.hist(tokens_count, bins='auto')  # arguments are passed to np.histogram
#plt.title("Histogram with 'auto' bins")
#plt.show()
# print len(tokens)
# print len(tokens_count)
# print max(tokens_count)
# print min(tokens_count)
print 'median counts: %d ' % np.median(tokens_count)
print 'frequency more than 20: %d' % len([tokens_count[i] for i in range(0, len(tokens_count))
                                                                         if tokens_count[i] > 19])
print 'number of unique characters: %d' % n_unique_token

median counts: 27 
frequency more than 20: 4052
number of unique characters: 7492


In [18]:
# Get the most common words and build index_to_word and word_to_index vectors
index_to_word = [x[0] for x in unique_token]
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

In [19]:
# Create the training data
X_train = np.asarray([[word_to_index[w] for w in poem[:-1]] for poem in poems_tokenize])
y_train = np.asarray([[word_to_index[w] for w in poem[1:]] for poem in poems_tokenize])

In [20]:
# Print an training data example
x_example, y_example = X_train[17], y_train[17]
print "x:\n%s\n%s" % (" ".join([index_to_word[x] for x in x_example]), x_example)
print "\ny:\n%s\n%s" % (" ".join([index_to_word[x] for x in y_example]), y_example)

x:
金 舆 巡 白 水 ， 玉 辇 驻 新 丰 。 纽 落 藤 披 架 ， 花 残 菊 破 丛 。 叶 铺 荒 草 蔓 ， 流 竭 半 池 空 。 纫 珮 兰 凋 径 ， 舒 圭 叶 翦 桐 。 昔 地 一 蕃 内 ， 今 宅 九 围 中 。 架 海 波 澄 镜 ， 韬 戈 器 反 农 。 八 表 文 同 轨 ， 无 劳 歌 大 风 。
[6531, 5085, 1596, 3996, 3030, 7488, 3729, 6262, 7114, 2409, 109, 76, 4698, 5332, 5460, 2121, 2646, 7488, 5157, 2970, 5292, 4167, 99, 76, 732, 6629, 5239, 5234, 5397, 7488, 3174, 4414, 646, 3050, 4367, 76, 4681, 3771, 445, 491, 1784, 7488, 5090, 1018, 732, 4883, 2727, 76, 2459, 1021, 80, 5412, 459, 7488, 186, 1366, 135, 997, 108, 76, 2646, 3207, 3124, 3414, 6677, 7488, 6939, 2037, 963, 707, 470, 76, 441, 5671, 2386, 742, 6235, 7488, 2432, 595, 2949, 1171, 6998, 76]

y:
舆 巡 白 水 ， 玉 辇 驻 新 丰 。 纽 落 藤 披 架 ， 花 残 菊 破 丛 。 叶 铺 荒 草 蔓 ， 流 竭 半 池 空 。 纫 珮 兰 凋 径 ， 舒 圭 叶 翦 桐 。 昔 地 一 蕃 内 ， 今 宅 九 围 中 。 架 海 波 澄 镜 ， 韬 戈 器 反 农 。 八 表 文 同 轨 ， 无 劳 歌 大 风 。 主
[5085, 1596, 3996, 3030, 7488, 3729, 6262, 7114, 2409, 109, 76, 4698, 5332, 5460, 2121, 2646, 7488, 5157, 2970, 5292, 4167, 99, 76, 732, 6629, 5239, 5234, 5397, 7488, 3174, 4414, 646, 305

In [ ]:
# Save training data
pickle.dump((X_train, y_train, index_to_word, word_to_index), open('trainingData.pkl', 'wb'))